In [4]:
#Kütüphaneler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


In [5]:
#Veriyi çekiyoruz
Data1=pd.read_excel("yorumlar.xlsx")

In [6]:
Data1

,Review,Liked
0,Wow... Loved this place,1
1,Crust is not good,0
2,Not tasty and the texture was just nasty,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone,0
997,Overall I was not impressed and would not go back,0
998,"The whole experience was underwhelming, and I ...",0


In [7]:
#Or
yorum=re.sub('[^a-zA-Z]',' ',Data1["Review"][1])

In [8]:
yorum

'Crust is not good'

In [9]:
yorum.lower()

'crust is not good'

In [10]:
yorum.split()

['Crust', 'is', 'not', 'good']

In [11]:
import nltk
from nltk.corpus import stopwords

In [12]:
from nltk.stem.porter import PorterStemmer

In [13]:
ps=PorterStemmer()


In [14]:
#Data1.drop([0],axis=0,inplace=True)
derleme=[]
for i in range(1000):
    yorum=re.sub("[^a-zA-Z]"," ",Data1["Review"][i])#Yorumdaki Harflerden haricindeki herşeyi kaldırıyoruz
    yorum=yorum.lower()#Bütün Harfleri küçültüyoruz
    yorum=yorum.split()#Kelimeler halinde ayiriyoruz
    yorum=[ps.stem(kelime) for kelime in yorum if not kelime in set(stopwords.words('english'))]#Ve bu kelimelerdeki StopWord(işe yaramayan) kelimeleri çıkartıyoruz.
    yorum=' '.join(yorum)# Geri cümle olarak Birleştiriyoruz
    derleme.append(yorum)# Cümleyi Listeye Ekliyoruz

In [15]:
#Kelimelerdeki Fiileri Köklerine ayırıyoruz
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
cv=CountVectorizer(max_features=2000) 
X=cv.fit_transform(derleme).toarray()
y=Data1.iloc[:,1].values

In [17]:
#Yorumlari train ve test olacak sekilde ayiriyoruz
from sklearn.model_selection import train_test_split

In [18]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=0)

In [19]:
#Naive bayes ile öğretip tahmin etiriyoruz
from sklearn.naive_bayes import GaussianNB

In [20]:
GNB=GaussianNB()
GNB.fit(x_train,y_train)
y_tahmin=GNB.predict(x_test)

In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
cm=confusion_matrix(y_test,y_tahmin)

In [23]:
cm

array([[ 87,  72],
       [ 26, 145]], dtype=int64)

In [24]:
#SVC
from sklearn.svm import SVC

In [25]:
svc=SVC()
svc.fit(x_train,y_train)
y_tahmin=svc.predict(x_test)
confusion_matrix(y_test,y_tahmin)

array([[146,  13],
       [ 77,  94]], dtype=int64)

In [26]:
#Karar ağacı
from sklearn.tree import DecisionTreeClassifier
DTC=DecisionTreeClassifier(criterion="entropy")
DTC.fit(x_train,y_train)
DTCtahmin=DTC.predict(x_test)
confusion_matrix(y_test,y_tahmin)

array([[146,  13],
       [ 77,  94]], dtype=int64)

In [35]:
#Rassal Orman
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=17,criterion="entropy")
rfc.fit(x_train,y_train)
rfcTahmin=rfc.predict(x_test)
confusion_matrix(y_test,rfcTahmin)

array([[137,  22],
       [ 70, 101]], dtype=int64)

In [36]:
#lojistik Regresyon
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(random_state=0)
lr.fit(x_train,y_train)
lrTahmin=lr.predict(x_test)
confusion_matrix(y_test,lrTahmin)

array([[132,  27],
       [ 60, 111]], dtype=int64)